In [37]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext, RDD
import math

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

1. Cree un RDD llamado lenguajes que contenga los siguientes lenguajes de programación: Python, R, C, Scala, Rugby y SQL.

    * a. Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en mayúsculas.

    * b. Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes de programación estén en minúsculas.

    * c. Cree un nuevo RDD que solo contenga aquellos lenguajes de programación que comiencen con la letra R.

In [16]:
# 1
lenguajes = sc.parallelize(["Python", "R", "C", "Scala", "Ruby", "SQL"])

In [17]:
# a
lenguajes_upper = lenguajes.map(lambda x: x.upper())
lenguajes_upper.collect()

['PYTHON', 'R', 'C', 'SCALA', 'RUBY', 'SQL']

In [18]:
# b
lenguajes_lower = lenguajes.map(lambda x: x.lower())
lenguajes_lower.collect()

['python', 'r', 'c', 'scala', 'ruby', 'sql']

In [19]:
# c
lenguajes_start_r = lenguajes.filter(lambda x: x.lower().startswith("r"))
lenguajes_start_r.collect()

['R', 'Ruby']

2. Cree un RDD llamado pares que contenga los números pares existentes en el intervalo [20;30].

    * a. Cree el RDD llamado sqrt, este debe contener la raíz cuadrada de los elementos que componen el RDD pares.

    * b. Obtenga una lista compuesta por los números pares en el intervalo [20;30] y sus respectivas raíces cuadradas. Un ejemplo del resultado deseado para el intervalo [50;60] sería la lista [50, 7.0710678118654755, 52, 7.211102550927978, 54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834].

    * c. Eleve el número de particiones del RDD sqrt a 20.

    * d. Si tuviera que disminuir el número de particiones luego de haberlo establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?

In [20]:
# 2
pares = sc.parallelize(list(range(20, 31, 2)))

In [27]:
# a
sqrt = pares.map(lambda x: math.sqrt(x))
sqrt.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [32]:
# b
sqrt_flatmap = pares.flatMap(lambda x: (x, math.sqrt(x)))
sqrt_flatmap.collect()

[20,
 4.47213595499958,
 22,
 4.69041575982343,
 24,
 4.898979485566356,
 26,
 5.0990195135927845,
 28,
 5.291502622129181,
 30,
 5.477225575051661]

In [33]:
# c
sqrt_20_partitions = sqrt.repartition(20)
sqrt_20_partitions.getNumPartitions()

20

In [34]:
# d
sqrt_10_partitions = sqrt.coalesce(10)
sqrt_10_partitions.getNumPartitions()

10

3. Cree un RDD del tipo clave valor a partir de los datos adjuntos como recurso a esta lección. Tenga en cuenta que deberá procesar el RDD leído para obtener el resultado solicitado. Supongamos que el RDD resultante de tipo clave valor refleja las transacciones realizadas por número de cuentas. Obtenga el monto total por cada cuenta.

* Tip: Cree su propia función para procesar el RDD leído.

In [48]:
rdd_kv = sc.textFile(r"../../data/transacciones")\
    .map(lambda x: x[1:-1].split(","))\
    .map(lambda x: (x[0], float(x[1].strip())))
rdd_kv.collect()

[('1001', 52.3),
 ('1005', 20.8),
 ('1001', 10.1),
 ('1004', 52.7),
 ('1005', 20.7),
 ('1002', 85.3),
 ('1004', 20.9)]

In [49]:
rdd_kv_reduce = rdd_kv.reduceByKey(lambda x, y: x + y)
sorted(rdd_kv_reduce.collect())

[('1001', 62.4), ('1002', 85.3), ('1004', 73.6), ('1005', 41.5)]